In [1]:
import os

In [2]:
dir='.'
# os.mkdir(dir)
# %rm <where BasicKernels/scripts is>/*

In [3]:
## versions=['1','2']
versions=['1','2']

for version in versions:
    if version == '1':
        xlas=['noxla']
    else:
        xlas=['xla','noxla']
    for xla in xlas:
        fn='rnn1d-tf'+version+'-'+xla+'.sh'
        print(fn)

        with open(os.path.join(dir,fn),'w') as f:
            f.write('''#!/bin/bash
#SBATCH -J {fn}\n'''.format(fn=fn.split('.')[0]))
            f.write('''#SBATCH -C gpu
#SBATCH --gres=gpu:1
#SBATCH --exclusive
#SBATCH -q special
#SBATCH -A m1759
#SBATCH -t 4:00:00

#activate env
module load tensorflow/{mod}
export PROFILER='cupy'
\n'''.format(mod='gpu-2.2.0-py37' if version=='2' else 'gpu-1.15.0-py37'))

            f.write('''#enalbe XLA or not, 'xla' or 'noxla'
export enable_xla='{xla}'

export XLA_FLAGS=--xla_gpu_cuda_data_dir=${{CUDA_HOME}}
export CUDA_DIR=${{CUDA_HOME}}
\n'''.format(xla=xla))
            
            f.write('''#rankspernode
rankspernode=1

#openmp  
export OMP_NUM_THREADS=$(( 40 / ${rankspernode} ))
export OMP_PLACES=threads
export OMP_PROC_BIND=spread
sruncmd="srun -N ${SLURM_NNODES} -n $(( ${SLURM_NNODES} * ${rankspernode} )) --cpu_bind=cores"
\n''')

            f.write('''#create run dir
run_dir=$PWD/tf_cnn_kernels_nsight/rnn1d-tf{version}-{xla}/$SLURM_JOBID/
mkdir -p ${{run_dir}}

#copy relevant files
script_dir=../python
script="rnn1d_tf{version}.py"
cp $script_dir/$script $run_dir/

#step in
cd ${{run_dir}}
\n'''.format(version=version,xla=xla))
        
            if version == '1':
                f.write('''if [ $enable_xla == "xla" ];then
    sed -i 's/allow_soft_placement=False/allow_soft_placement=True/g' $script
fi
\n''')

            f.write('''#net_params
net_params="16,16,32,16 32,16,32,16, 64,16,32,16 128,16,32,16 "
net_params+="16,32,32,16 16,64,32,16 16,128,32,16 "
net_params+="16,16,64,16 16,16,128,16 "
net_params+="16,16,32,32 16,16,32,64 16,16,32,128 "

#list of metrics
#metrics="sm__cycles_elapsed.avg "
metrics="sm__cycles_elapsed.avg.per_second,\\
sm__cycles_elapsed.avg,\\
sm__inst_executed_pipe_tensor.sum,\\
sm__sass_thread_inst_executed_op_fadd_pred_on.sum,\\
sm__sass_thread_inst_executed_op_ffma_pred_on.sum,\\
sm__sass_thread_inst_executed_op_fmul_pred_on.sum,\\
sm__sass_thread_inst_executed_op_hadd_pred_on.sum,\\
sm__sass_thread_inst_executed_op_hfma_pred_on.sum,\\
sm__sass_thread_inst_executed_op_hmul_pred_on.sum,\\
dram__bytes.sum,\\
lts__t_bytes.sum,\\
l1tex__t_bytes.sum "

#export TF_XLA_FLAGS="--tf_xla_auto_jit=2"
#export XLA_FLAGS="--xla_dump_to=$run_dir"

cells="lstm"
precs="16 32"

num_warmup=5
num_iter=1

for prec in $precs; do
    for cell in $cells; do
        for net_param in ${net_params}; do
            tmp_param=(${net_param//,/ })
            batch=${tmp_param[0]}
            time=${tmp_param[1]}
            feature=${tmp_param[2]}
            h_size=${tmp_param[3]}
            input_tensor_shape=$batch" "$time" "$feature
            echo $input_tensor_shape
\n''')
                
            f.write('''            outputstr=tf{version}.fp_${{prec}}.celltype_${{cell}}.input_${{batch}}x${{time}}x${{feature}}.nneu_${{h_size}}\n'''.format(version=version))
            f.write('''            #iterate over FW BW
            for ctype in calibrate forward backward; do

                #profile string
                profilestring="/usr/common/software/cuda/11.0.167/bin/nv-nsight-cu-cli \\
                --profile-from-start off --metrics ${metrics} --kernel-base demangled -o \\
                ${outputstr}.pass_${ctype}.${enable_xla}"

                if [ $enable_xla == "xla" ];then
                    ${sruncmd} ${profilestring} $(which python) -u ./$script \\
                        --input_tensor_shape ${input_tensor_shape} \\
                        --cell_type ${cell} \\
                        --n_neurons ${h_size} \\
                        --dtype float${prec} \\
                        --num_iterations ${num_iter} \\
                        --num_warmups ${num_warmup} \\
                        --enable_xla \\
                        --compute_type ${ctype}
                else
                    ${sruncmd} ${profilestring} $(which python) -u ./$script \\
                        --input_tensor_shape ${input_tensor_shape} \\
                        --cell_type ${cell} \\
                        --n_neurons ${h_size} \\
                        --dtype float${prec} \\
                        --num_iterations ${num_iter} \\
                        --num_warmups ${num_warmup} \\
                        --compute_type ${ctype}
                fi
            done
        done
    done
done\n''')



rnn1d-tf1-noxla.sh
rnn1d-tf2-xla.sh
rnn1d-tf2-noxla.sh
